In [ ]:
!pip install --upgrade pip
!pip install opencv-python
!pip install opencv-contrib-python
!pip install matplotlib
!pip install darknetpy

In [2]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt 

In [3]:
net = cv2.dnn.readNet("../input/objectdetection/yolov3.weights", "../input/objectdetection/yolov3.cfg")

classes = []

with open("../input/objectdetection/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

error: OpenCV(4.5.2) /tmp/pip-req-build-eirhwqtr/opencv/modules/dnn/src/darknet/darknet_importer.cpp:207: error: (-212:Parsing error) Failed to parse NetParameter file: ../input/objectdetection/yolov3.cfg in function 'readNetFromDarknet'


In [ ]:
img = cv2.imread("../input/objectdetection/table.jpg")
img = cv2.resize(img, None, fx=0.4, fy=0.4)

height, width, channels = img.shape

In [ ]:
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

net.setInput(blob)
outs = net.forward(output_layers)

In [ ]:
class_ids = []
confidences = []
boxes = []

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        
        if confidence >= 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

In [ ]:
%matplotlib inline

indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y + 30), font, 3, color, 3)
        
plt.imshow(img)
plt.show()

cv2.imwrite("./objectified-table.jpg", img)
cv2.waitKey(0)
cv2.destroyAllWindows()